# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps.datasets 
import json
from api_keys import g_key
from pprint import pprint
import time


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_filepath = os.path.join("..", "VacationPy", "cities.csv")
heatmap_df = pd.read_csv(csv_filepath)
heatmap_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps + Create heatmap
gmaps.configure(api_key = g_key)

fig = gmaps.figure()
locations = heatmap_df[["Lat", "Lng"]]
weights = heatmap_df["Humidity"]
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights = weights, point_radius = 5, max_intensity = 50))

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#Narrow Down DF via Weather Conditions to Find Ideal Locations

hotel_df = heatmap_df.loc[(heatmap_df["Max Temp"] > 75) & (heatmap_df["Max Temp"]< 95) & (heatmap_df["Wind Speed"] < 10) & (heatmap_df["Cloudiness"] == 0), ["City", "Cloudiness", "Country", "Humidity", "Lat", "Lng", "Max Temp", "Wind Speed"]]
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location="
hotel_name = []

for index, row in hotel_df.iterrows():
    query_url = base_url + str(row["Lat"]) + "," + str(row["Lng"]) + "&radius=5000&type=lodging&key=" + g_key
    response = requests.get(query_url).json()
    time.sleep(0.5)
    try:
        hotel_name.append(response["results"][0]["name"])
    except: 
        hotel_name.append("Not Found")
        pass
    
hotel_df["Hotel Name"] = hotel_name
hotel_df

,City,Cloudiness,Country,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
26,colonelganj,0,IN,14,27.13,81.70,94.20,5.08,सूरज होटल कर्नलगंज लखनऊ रोड
88,cidreira,0,BR,66,-30.17,-50.22,76.56,5.50,Pond Country Club
125,arraial do cabo,0,BR,78,-22.97,-42.02,84.20,4.70,Pousada Porto Praia
138,morondava,0,MG,69,-20.30,44.28,79.08,9.06,La Case Bambou
139,vaini,0,IN,88,15.34,74.49,78.90,1.59,Dandeli Lake County
142,sarai mir,0,IN,35,26.02,82.92,86.00,3.36,Haji Munshi Kahn Read Sherwan
176,nador,0,MA,40,35.17,-2.93,75.20,6.93,Hôtel La Giralda
244,pasni,0,PK,52,25.26,63.48,86.82,9.17,PHE Rest House
316,gunjur,0,GM,61,13.18,-16.76,80.60,9.17,Boboi Beach Lodge
416,kirkuk,0,IQ,16,35.47,44.40,86.82,7.36,Bayaz Han (Beyaz Han)


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# Generate new Hotel Map with Plot Markers

filtered_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "Not Found", ["Hotel Name", "City", "Country", "Lat", "Lng"]]

hotel_markers_list = []
hotel_markers_dict = {}

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name

for index, row in filtered_hotel_df.iterrows():
    hotel_markers_dict["Hotel Name"] = row["Hotel Name"]
    hotel_markers_dict["location"] = (row["Lat"], row["Lng"])
    hotel_markers_dict["City"] = row["City"]
    hotel_markers_dict["Country"] = row["Country"]
    hotel_markers_list.append(hotel_markers_dict)
    hotel_markers_dict = {}
    
# Using the template add the hotel marks to the heatmap

city_locations = [city["location"] for city in hotel_markers_list]
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Add marker layer ontop of heat map

hotel_info = [info_box_template.format(**city) for city in hotel_markers_list]
marker_layer = gmaps.marker_layer(city_locations, info_box_content = hotel_info)

# Display Map

fig = gmaps.figure()
fig.add_layer(marker_layer)
fig.add_layer(gmaps.heatmap_layer(locations, weights = weights, point_radius = 10, max_intensity = 200))

fig

Figure(layout=FigureLayout(height='420px'))